In [1]:
import requests
import json
import pandas as pd
import random

In [107]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

In [2]:
SURVEY_ID = 709692367

In [21]:
AUTH_URL = "https://akvofoundation.eu.auth0.com/oauth/token"
SURVEY_URL = f"https://api-auth0.akvo.org/flow/orgs/maep/surveys/{SURVEY_ID}"
MONITORING_FORM_NAME = "2_INFORMATIONS SUR L’EXPLOITATION ET LA PARCELLE (Formulaire de suivi)"

In [4]:
USERNAME = ""
PASSWORD = ""

In [5]:
r = requests.Session()

In [6]:
metadata_keys = ['formVersion',
 'deviceIdentifier',
 'dataPointId',
 'submissionDate',
 'modifiedAt',
 'id',
 'identifier',
 'displayName',
 'formId',
 'surveyalTime',
 'submitter',
 'createdAt']

In [7]:
def get_token():                                                                                                                                                     
    data = {                                                                                                                                                                                                         
        "client_id": "S6Pm0WF4LHONRPRKjepPXZoX1muXm1JS",                                                                                                                                                             
        "username": USERNAME,                                                                                                                                                                                        
        "password": PASSWORD,                                                                                                                                                                     
        "grant_type": "password",                                                                                                                                                                                    
        "scope": "offline_access"                                                                                                                                                                                    
    }                                                                                                                                                                                                                
    req = r.post(AUTH_URL, data=data)                                                                                                                                                                             
    if req.status_code != 200:                                                                                                                                                                                       
        print("LOGIN ERROR")                                                                                                                                                              
    return req.json()

In [8]:
def get_data(target_url, headers):
    # print(f"GET: {target_url}")
    res = r.get(target_url, headers=headers)
    if res.status_code != 200:                                                                                                                                                                                       
        print("FETCH DATA ERROR")
    return res.json()

In [9]:
def get_headers(refresh_token):                                                                             
    login = {                                                                                            
        'client_id': 'S6Pm0WF4LHONRPRKjepPXZoX1muXm1JS',                                                 
        'grant_type': 'refresh_token',                                                                   
        'refresh_token': refresh_token,                                                                          
        'scope': 'openid email'                                                                          
    }                                                                                                    
    req = r.post(AUTH_URL, data=login)                          
    if req.status_code != 200:                                                                           
        return False                                                                                     
    return {                                                                                             
        'Content-Type': 'application/json',                                                              
        'Accept': 'application/vnd.akvo.flow.v2+json',                                                   
        'Authorization': 'Bearer {}'.format(req.json().get('id_token'))                                  
    }

In [12]:
def fetch_all(url, headers, formInstances=[]):                                                                                                                                                                       
    data = get_data(url, headers)                                                                                                                                                                                    
    next_url = data.get('nextPageUrl')                                                                                                                                                                               
    data = data.get('formInstances')
    # LIMITING DATA TO 100
    # if len(formInstances) == 100:
    #    print("FINISH THE LIMIT")
    #   return formInstances
    # END TEMPORARY CODE
    if data:                                                                                                                                                                                                         
        for d in data:                                                                                                                                                                                               
            formInstances.append(d)                                                                                                                                                                                  
        if next_url:                                                                                                                                                                                                 
            fetch_all(next_url, headers, formInstances)                                                                                                                                                              
    return formInstances 

In [13]:
def data_handler(data, qType):                                                                                                                                                                                       
    if data:                                                                                                                                                                                                         
        if qType in [                                                                                                                                                                                                
                'FREE_TEXT', 'NUMBER', 'BARCODE', 'DATE', 'GEOSHAPE', 'SCAN',                                                                                                                                        
                'CADDISFLY'                                                                                                                                                                                          
        ]:                                                                                                                                                                                                           
            return data                                                                                                                                                                                              
        if qType == 'OPTION':                                                                                                                                                                                        
            return handle_list(data, "text")                                                                                                                                                                         
        if qType == 'CASCADE':                                                                                                                                                                                       
            return handle_list(data, "name")                                                                                                                                                                         
        if qType in ['PHOTO', 'VIDEO']:                                                                                                                                                                              
            return data.get('filename')                                                                                                                                                                              
        if qType == 'VIDEO':                                                                                                                                                                                         
            return data.get('filename')                                                                                                                                                                              
        if qType == 'GEO':                                                                                                                                                                                           
            return {'lat': data.get('lat'), 'long': data.get('long')}                                                                                                                                                
        if qType == 'SIGNATURE':                                                                                                                                                                                     
            return data.get("name")                                                                                                                                                                                  
    return None

In [14]:
def handle_list(data, target):                                                                                                                                                                                       
    response = []                                                                                                                                                                                                    
    for value in data:                                                                                                                                                                                               
        response.append(value.get(target))                                                                                                                                                                       
    return "|".join(response)

In [15]:
def get_page(form_instance_url, form_definition, refresh_token):                                                                                                                                               
    headers = get_headers(refresh_token)                                                                                                                                                                        
    collections = fetch_all(form_instance_url, headers)                                                                                                                                                          
    results = []                                                                                                                                                                                                     
    for col in collections:                                                                                                                                                                                          
        dt = {}                                                                                                                                                                                                      
        for c in col:                                                                                                                                                                                                
            if c != 'responses':                                                                                                                                                                                     
                dt.update({c: col[c]})                                                                                                                                                                               
            else:                                                                                                                                                                                                    
                for g in form_definition:                                                                                                                                                                            
                    answers = col.get(c)                                                                                                                                                                             
                    answers = answers.get(g['id']) if answers else None
                    for q in g['questions']:                                                                                                                                                                         
                        d = None
                        try:
                            a = answers[0].get(q['id'])
                            d = data_handler(a, q['type'])                                                                                                                                                           
                        except Exception as e:                                                                                                                                                                       
                            pass                                                                                                                                                                         
                        # n = "{}|{}".format(q['id'], q['name'])                                                                                                                                                       
                        dt.update({q['name']: d})                                                                                                                                                                            
        results.append(dt)                                                                                                                                                                                           
    return results

In [16]:
refresh_token = get_token().get("refresh_token")
headers = get_headers(refresh_token)
surveys = get_data(SURVEY_URL, headers)

GET: https://api-auth0.akvo.org/flow/orgs/maep/surveys/709692367


In [17]:
registration_form_id = surveys.get("registrationFormId")
monitoring_forms = list(filter(lambda x: x.get("id") != registration_form_id, surveys.get("forms")))

In [18]:
data = {}

In [19]:
for form in surveys.get("forms"):
    form_id = form.get("id")
    form_definition = form.get("questionGroups")
    collections = get_page(form.get("formInstancesUrl"), form_definition, refresh_token)
    data.update({form_id: collections})

GET: https://api-auth0.akvo.org/flow/orgs/maep/form_instances?survey_id=709692367&form_id=715752508
GET: https://api-auth0.akvo.org/flow/orgs/maep/form_instances?survey_id=709692367&form_id=715752508&cursor=CjASKmoOZX5ha3ZvZmxvdy0xODJyGAsSDlN1cnZleUluc3RhbmNlGNeZpMcCDBgAIAA
GET: https://api-auth0.akvo.org/flow/orgs/maep/form_instances?survey_id=709692367&form_id=715752508&cursor=CjASKmoOZX5ha3ZvZmxvdy0xODJyGAsSDlN1cnZleUluc3RhbmNlGMvm2skCDBgAIAA
GET: https://api-auth0.akvo.org/flow/orgs/maep/form_instances?survey_id=709692367&form_id=715752508&cursor=CjASKmoOZX5ha3ZvZmxvdy0xODJyGAsSDlN1cnZleUluc3RhbmNlGOHK-s4CDBgAIAA
GET: https://api-auth0.akvo.org/flow/orgs/maep/form_instances?survey_id=709692367&form_id=715752508&cursor=CjASKmoOZX5ha3ZvZmxvdy0xODJyGAsSDlN1cnZleUluc3RhbmNlGMSn09ACDBgAIAA
GET: https://api-auth0.akvo.org/flow/orgs/maep/form_instances?survey_id=709692367&form_id=715752508&cursor=CjASKmoOZX5ha3ZvZmxvdy0xODJyGAsSDlN1cnZleUluc3RhbmNlGPH3zNECDBgAIAA
GET: https://api-auth0.ak

In [22]:
def get_monitoring_id_by_name(name:str):
    target = list(filter(lambda x: x.get("name").lower() == name.lower(), monitoring_forms))
    if len(target):
        return target[0].get("id")
    else:
        print(f"NOT FOUND: {name}")

In [ ]:
## DATA COLLECTIONS ENDS HERE
# WE ARE GOING TO TRANSFORM HERE

In [51]:
monitoring_form_id = get_monitoring_id_by_name(MONITORING_FORM_NAME)
registration_data = data.get("715752508")
monitoring_data = data.get(monitoring_form_id)

In [114]:
registration = pd.DataFrame(registration_data)
monitoring = pd.DataFrame(monitoring_data)

In [115]:
def splitter(x, pos):
    administration = x.split("|")
    if pos > len(administration):
        return ""
    return  administration[pos]

def extract_area(x):
    if x != x:
        return 0
    area_list = x.get("features")
    area = 0
    if area_list:
        for a in area_list:
            subarea = float(a.get("properties").get("area").replace(",","."))
            area += subarea
    return area

In [116]:
registration["Pole"] = registration["Localisation administrative"].apply(lambda x: splitter(x, 0))
registration["Département"] = registration["Localisation administrative"].apply(lambda x: splitter(x, 1))
registration["Commune"] = registration["Localisation administrative"].apply(lambda x: splitter(x, 2))
registration["Arrondissement"] = registration["Localisation administrative"].apply(lambda x: splitter(x, 3))
registration["Village"] = registration["Localisation administrative"].apply(lambda x: splitter(x, 4))
registration["AREA"] = registration["Superficie mesurée de la parcelle (smartphone)"].apply(extract_area)

In [117]:
merged = registration.merge(monitoring, on='dataPointId', suffixes=('_registration', '_monitoring'))

In [124]:
aggregation = merged.groupby(["Commune"], as_index = False)["AREA"].agg(
    ['min','mean','max', 'sum'])

In [125]:
aggregation

,min,mean,max,sum
Commune,,,,
ADJA OUERE,117.521439,507.107109,896.692780,1.014214e+03
BANIKOARA,2.072357,272.464086,480.111511,1.634785e+03
BANTE,0.010000,296.479930,998.768082,3.261279e+03
BASSILA,0.000000,181.104914,597.062744,2.535469e+03
COPARGO,0.389343,731.042938,1461.696533,1.462086e+03
DASSA ZOUME,3.641281,16762.794660,53989.594879,7.543258e+05
DJIDJA,84.152298,162703.379179,645725.605316,1.301627e+07
DJOUGOU,98.547745,735.677158,2087.125900,3.678386e+03
GLAZOUE,0.000000,64.301392,167.238983,1.929042e+02


In [126]:
registration.groupby(["Pole","Département","Commune","Arrondissement"], as_index = False)["AREA"].agg(
    ['mean','sum'])

mean           sum
Pole Département Commune     Arrondissement                                   
PDA1 ALIBORI     KARIMAMA    BIRNI LAFIA                0.000000  0.000000e+00
                             KARIMAMA                 126.481842  2.529637e+02
PDA2 ALIBORI     BANIKOARA   BANIKOARA                480.111511  4.801115e+02
                             FOUNOUGO                 117.894318  2.357886e+02
                             SOROKO                   436.700500  4.367005e+02
                 GOGOUNOU    GOGOUNOU                 873.683044  8.736830e+02
                             SORI                     433.485000  8.669700e+02
                 KANDI       KANDI 1                  652.588409  1.305177e+03
                 SEGBANA     SEGBANA                    3.234924  3.234924e+00
     ATACORA     KEROU       KEROU                    580.257660  5.802577e+02
                 KOUANDE     BIRNI                   1422.239990  2.844480e+03
                             KOUANDE                  111.293671  1.112937e+02
     BORGOU      KALALE      KALALE                   522.636536  1.045273e+03
                             PEONGA                     5.829376  5.829376e+00
                 SINENDE     F BOURE                   44.226227  4.422623e+01
                             SINENDE                   99.017944  9.901794e+01
PDA3 ATACORA     MATERI      MATERI                     0.180756  1.807556e-01
PDA4 BORGOU      NDALI       BORI                      66.972260  6.697226e+01
                             N DALI                 30160.211243  6.032042e+04
                             SIRAROU                  858.204956  8.582050e+02
                 NIKKI       NIKKI                    111.114858  3.333446e+02
                 PARAKOU     1er ARRONDISSEMENT         2.236725  2.236725e+00
                             2ème Arrondissement     1230.082825  1.230083e+03
                             3ème Arrondissement      289.963638  1.159855e+03
                 PERERE      GUINAGOUROU              184.803284  5.544099e+02
                 TCHAOUROU   ALAFIAROU               1204.135864  1.204136e+03
                             BETEROU                  139.651077  2.793022e+02
                             KIKA                      43.232025  4.323203e+01
                             SANSON                  3347.632243  1.004290e+04
                             TCHAOUROU               1718.015945  6.872064e+03
                             TCHATCHOU                  0.434845  4.348450e-01
     COLLINES    BANTE       BANTE                      0.085000  1.700000e-01
                             BOBE                     554.914299  2.219657e+03
                             GOUKA                    933.912033  9.339120e+02
                             KOKO                       0.010000  1.000000e-02
                 DASSA ZOUME AKOFFODJOULE           25570.944094  4.602770e+05
                             DASSA II                8671.586398  1.387454e+05
                             DASSA I                39831.969727  3.983197e+04
                             LEMA                       3.900162  3.900162e+00
                             PAOUINGNAN               670.037796  6.700378e+02
                             TRE                     5032.597546  2.013039e+04
                 GLAZOUE     GLAZOUE                    0.000000  0.000000e+00
                             GOME                     167.238983  1.672390e+02
                             ZAFFE                     25.665192  2.566519e+01
                 OUESSE      GBANLIN                  853.121979  8.531220e+02
                             KILIBO                  1449.153168  1.449153e+03
                             OUESSE                   284.746750  5.694935e+02
                 SAVALOU     KPATABA                30251.601589  1.754593e+06
                             MONKPA                     0.000000  0.000000e+00
                             SA